For the task, I have decided to try the RL methods on the "taxi-v3" game. In this part, I have followed the Q-learning method to implement this. 
The rules of the game are as followed: 

In [9]:
%pip install gym
%pip install pygame
#I started by installing the necessary packages, which is gym and pygame. Gym will be used to create and interact with the envirnoment for the taxi game.
#pygame package is used in order to visualize the environemnt, while the first time running the proceeding code (to create the environment), it showed an error that it couldn't  reate the visualization of the game and hence it was suggested to import pygame as well to solve this bug.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import random
import gym
import pygame
import time
from IPython.display import clear_output

#here, I have imported all the libraries - numpy for arrays, random, gymnasium to create the environment, pygame, time for , IPython.display

In [11]:
env=gym.make("Taxi-v3",render_mode='ansi')
#this line of code is used to create the environment for the taxi game, we have used the function gym.make and we have rendered it in "ansi" mode which means that it will execute the visualization of the game for the human reader in text-based form 
#I also tried "human", for 'render_mode', but it was slow and not working at the best efficiency because it is very resource heavy hence I decided to opt for "ansi" mode instead as it is more lightweight.

In [12]:
#now the following code is used to craeate a Q-table - a Q-table is used to store the Q-values for each state-action pair
action_size=env.action_space.n #retreiving the number of actions in the action set 'A', this will be used to create the number of rows.
print("Action size", action_size)
#number possible actions possible - 6(up, down, left, right, drop person, pickup person)

state_size=env.observation_space.n #retrieving the number of different elements in the States 'S' set, this will be used for the number of columnns
print("State size",state_size)
#the different 

Action size 6
State size 500


In [13]:
#here all the values in the Q-table are being initialized to zero, this allows the agent to explore the environment and update the q-values ccordingly so that it can exploit the evironment.
qtable=np.zeros((state_size,action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [14]:
#here we are defining the different variables
total_episodes=1000 #this is the number of episodes used to train the model
total_test_episodes=100 #this is the total number of episodes used to test the model 
max_steps=100 #this is the maximum number of steps per episodes

learning_rate=0.3 #this is the learing rate - alpha - for updating the Q-values
discount_rate=0.625 #this is the discount rate that is used for future rewards

exploration_rate=1.0 #this is the initial exploration rate for teh epsilon-greedy strategy
max_exploration_rate=1.0 #min and max exploration rates are the limits for the rate
min_exploration_rate=0.01
decay_rate=0.01 #decay rate for the exploration rate - how quickly it decays over time

In [15]:
rewards_all_episodes=[] #to store all the reawrds collected in each episode

In [16]:
for episode in range(total_episodes): #loop to run it 1000 times, which is also = to the total number of episodes for training
    state=env.reset()[0] #this resets the environment entirely and sets the state variable to the the first element of the returned tuple
    done=False #done represents whether the episode is finished or not yet
    rewards_current_episode=0 #to keep trak of the rewards within the current episode, iti s set to 0 as we start off each episode with no rewards 

    for step in range(max_steps):

        exploration_rate_threshold=random.uniform(0,1) #here we have used a function from the random library to generate a random number that will decide whether teh agent will explore or exploit
        if exploration_rate_threshold>exploration_rate: #if the randomly generated number is greater than the exploration rate, then the agent will exploit the environment
            action=np.argmax(qtable[state,:]) #the agent chooses the action with the highest Q-value for the next action (thus exploiting the information it has)
        else:
            action=env.action_space.sample() #this chooses a random action in space to perform as the agent is exploring

        new_state, reward, done, truncated, info = env.step(action) #after choosing the action to perform, this line will actually execute the action and retrieve the info regarding the new state, the rewards, whether the episode is done or not, whether the episode has been truncated, and any additional info 

        qtable[state,action]=qtable[state,action]*(1-learning_rate)+learning_rate*(reward+discount_rate+np.max(qtable[new_state, :])) #this updates the Q value for the particular state-action pair in the q-table according to equation

        state=new_state #updates the current step to the new state
        rewards_current_episode+=reward 

        if done==True: #if the episode is done, the loop is broken
            break

    

    exploration_rate=min_exploration_rate+(max_exploration_rate-min_exploration_rate)*np.exp(-decay_rate*episode) #this line of code here, updates the exploration rate according to th eparticular equation so that over time, the exploration rate is decreased thus gradually changing exploration to exploitation

    rewards_all_episodes.append(rewards_current_episode) #so this line of code, appends the the rewards in each episode - the rewards in the current episode to a list named "rewards_all_episodes"


rewards_per_thousand_episodes=np.split(np.array(rewards_all_episodes),total_episodes/1000) #This splits the rewards into chunks of 1000 episodes each. Since total_episodes is 1000, it results in one chunk.

count=1000

print("Average rewards per thousand episodes\n") #the block of code calculates and finds out the average rewards per thousand episodes, iterates ovver each chunk and computes the average rewards
for r in rewards_per_thousand_episodes:
    print(count,":",str(sum(r/1000)))
    count+=1000

print("\n\n******Q-table******\n")
print(qtable) #printing the final q-table with the updated values

Average rewards per thousand episodes

1000 : -55.33799999999997


******Q-table******

[[ 0.          0.          0.          0.          0.          0.        ]
 [-1.84607645 -1.76073166 -1.87374241 -1.71938203 17.24493554 -8.87239265]
 [-1.03855559 -0.99888289 -1.12395053 -0.96366375 18.74999902 -2.71540459]
 ...
 [-0.552375   19.12499964 -0.63844875 -0.59335399 -4.815      -2.07688129]
 [-1.0125     -0.95851149 -0.97875    -1.04533027 -6.3855     -2.8125    ]
 [-0.225      -0.225      -0.225      13.875345   -2.8125     -2.8125    ]]


In [17]:
#for this part of the code, we are trying to observe the performance of the agent in the environment, hence I created a loop to run the agent's performance 3 times
for episode in range(3): 
    state = env.reset()[0] 
    done=False
    print("Episode ",episode+1,"\n\n\n\n")
    time.sleep(1) #pauses the execution for 1 second

    for step in range(max_steps):
        clear_output(wait=True) #Clears the output in the current cell, particularly used in java
        print(env.render()) #to render the final state of the environment in ansi form
        time.sleep(0.3)

        action=np.argmax(qtable[state,:])
        new_state, reward, done, truncated, info=env.step(action)

        if done:
            clear_output(wait=True)
            print(env.render())
            if reward==20:
                print("You reached the goal")
                time.sleep(3)
            else:
                print("You didnt")
                time.sleep(3)
            clear_output(wait=True)
            break

        state=new_state

env.close() #closes the environment 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

You reached the goal
